In [2]:
import os
import pandas as pd

# ——— 1. Set the data directory ———
# —It's inside a notebooks/ subdirectory, use:
data_dir = os.path.abspath(os.path.join(os.getcwd(), '..', 'data'))

# ——— 2. Read the CSV files ———
doctors      = pd.read_csv(os.path.join(data_dir, 'doctors.csv'))
appointments = pd.read_csv(os.path.join(data_dir, 'appointments.csv'))
treatments   = pd.read_csv(os.path.join(data_dir, 'treatments.csv'))
billing      = pd.read_csv(os.path.join(data_dir, 'billing.csv'))
patients     = pd.read_csv(os.path.join(data_dir, 'patients.csv'))

# ——— 3. Fill missing values ———
# Replace missing specializations, cost, and billing amounts
doctors['specialization'] = doctors['specialization'].fillna('Unknown')
treatments['cost']        = treatments['cost'].fillna(treatments['cost'].median())
billing['amount']         = billing['amount'].fillna(billing['amount'].median())

# ——— 4. Parse appointment_date and extract quarter ———
appointments['appointment_date'] = pd.to_datetime(
    appointments['appointment_date'], errors='coerce'
)
appointments['quarter'] = appointments['appointment_date'].dt.to_period('Q')

# ——— 5. Compute doctor FTE ———
# Treat each doctor record as 1 FTE, then aggregate by branch
doctors['fte'] = 1
branch_fte = (
    doctors
    .groupby('hospital_branch')
    .agg(fte_doctors=('fte', 'sum'))
    .reset_index()
)

# ——— 6. Summarize billed amounts by branch and quarter ———
# Step 1: Link billing to treatments (which contains appointment_id)
bill_treat = billing.merge(
    treatments[['treatment_id', 'appointment_id']],
    on='treatment_id', how='left'
)
# Step 2: Link to appointments (which contains doctor_id and quarter)
bill_treat_appt = bill_treat.merge(
    appointments[['appointment_id', 'doctor_id', 'quarter']],
    on='appointment_id', how='left'
)
# Step 3: Link to doctors (which contains hospital_branch)
bill_treat_appt_doc = bill_treat_appt.merge(
    doctors[['doctor_id', 'hospital_branch']],
    on='doctor_id', how='left'
)
# Final aggregation of billed amount
billing_summary = (
    bill_treat_appt_doc
    .groupby(['hospital_branch', 'quarter'])['amount']
    .sum()
    .reset_index(name='total_billed_amount')
)

# ——— 7. Summarize patient days by branch and quarter ———
appt_doc = appointments.merge(
    doctors[['doctor_id', 'hospital_branch']],
    on='doctor_id', how='left'
)
patient_summary = (
    appt_doc
    .groupby(['hospital_branch', 'quarter'])['patient_id']
    .nunique()
    .reset_index(name='patient_days')
)

# ——— 8. Summarize treatment count by branch and quarter ———
treatment_summary = (
    bill_treat_appt_doc
    .groupby(['hospital_branch', 'quarter'])
    .size()
    .reset_index(name='treatment_count')
)

# ——— 9. Generate all branch × quarter combinations ———
branches = branch_fte['hospital_branch'].unique()
quarters = appointments['quarter'].dropna().unique()
branch_quarters = pd.DataFrame(
    [(b, q) for b in branches for q in quarters],
    columns=['hospital_branch', 'quarter']
)

# ——— 10. Merge all summaries into one table ———
summary = branch_quarters.merge(branch_fte, on='hospital_branch', how='left')
summary = summary.merge(treatment_summary,    on=['hospital_branch','quarter'], how='left')
summary = summary.merge(patient_summary,      on=['hospital_branch','quarter'], how='left')
summary = summary.merge(billing_summary,      on=['hospital_branch','quarter'], how='left')

# ——— 11. Fill missing values and calculate efficiency ———
summary['treatment_count']     = summary['treatment_count'].fillna(0)
summary['patient_days']        = summary['patient_days'].fillna(0)
summary['total_billed_amount'] = summary['total_billed_amount'].fillna(0)
summary['efficiency'] = summary.apply(
    lambda row: row['treatment_count'] / row['patient_days']
                if row['patient_days'] > 0 else 0,
    axis=1
)

# ——— 12. Save the final summary ———
output_path = os.path.join(data_dir, 'hospital_staffing_summary.csv')
summary.to_csv(output_path, index=False)

# ——— 13. Display a preview ———
print("✅ Week 2: Data cleaning and feature engineering complete. Summary preview:")
display(summary.head())
print(f"\nSaved summary to: {output_path}")


✅ Week 2: Data cleaning and feature engineering complete. Summary preview:


,hospital_branch,quarter,fte_doctors,treatment_count,patient_days,total_billed_amount,efficiency
0,Central Hospital,2023Q3,4,19,17,53407.13,1.117647
1,Central Hospital,2023Q2,4,27,22,73027.57,1.227273
2,Central Hospital,2023Q1,4,27,22,69543.41,1.227273
3,Central Hospital,2023Q4,4,11,10,33061.33,1.100000
4,Eastside Clinic,2023Q3,3,11,10,24002.07,1.100000



Saved summary to: c:\Users\Admin\OneDrive - Yeshiva University\Desktop\Capstone\data\hospital_staffing_summary.csv
